# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [01:59<00:00, 23.94s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

"Title: Energizer LED Pen Light for $4 + free shipping w/ $35\nDetails: It's the best price we could find by $3. Buy Now at Walmart\nFeatures: \nURL: https://www.dealnews.com/products/Energizer/Energizer-LED-Pen-Light/495854.html?iref=rss-c196"

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: JBL Early Access Black Friday Sale: Up to 70% off + free shipping
Details: JBL discounts a huge range of headphones, portable speakers, soundbars, and more for their Labor Day sale, with deals starting from $10. Plus, everything ships for free. Shop Now at JBL
Features: 
URL: https://www.dealnews.com/JBL-Early-Access-Black-Friday-Sale-Up-to-70-off-free-shipping/2178

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
len(result.deals)

5

In [12]:
result.deals[1]

Deal(product_description="The Bose SoundLink Flex Bluetooth Speaker delivers exceptional sound quality in a compact and rugged design. With an IP67 waterproof rating and up to 12 hours of battery life, it's perfect for outdoor use. This speaker also features a built-in microphone for hands-free calls and a durable construction, making it ideal for beach trips, picnics, or any adventure. Its portability and rich sound performance make it a must-have for music lovers on the go.", price=79.0, url='https://www.dealnews.com/products/Bose/Bose-Sound-Link-Flex-Bluetooth-Speaker/477740.html?iref=rss-c142')

In [13]:
from agents.scanner_agent import ScannerAgent

In [14]:
agent = ScannerAgent()
result = agent.scan()

In [15]:
result

DealSelection(deals=[Deal(product_description='The EcoFlow Delta Pro is a powerful 3,600Wh portable power station that comes with an additional 3,600Wh battery and a 220W solar panel. This versatile energy solution is designed to power multiple devices and appliances, making it ideal for outdoor adventures or emergency backup. With the capability to charge via solar or traditional outlet, it offers convenience and efficiency for users looking to harness renewable energy. The system is robust and reliable, ensuring that you have power when you need it most.', price=1999.0, url='https://www.dealnews.com/Certified-Refurb-Eco-Flow-Delta-Pro-3-600-Wh-Power-Station-Extra-3-600-Wh-220-W-Solar-Panel-for-1-999-free-shipping/21784054.html?iref=rss-c142'), Deal(product_description='The Certified Refurb Bose SoundLink Flex Bluetooth Speaker is a compact and durable audio device designed for excellent sound quality on the go. With an IP67 waterproof rating, it can withstand splashes and spills, mak